In [4]:
# !pip install vaderSentiment
# !pip install imblearn
# !pip install xgboost

In [101]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.metrics import confusion_matrix, classification_report, plot_confusion_matrix
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [10]:
# Health & Personal Care Category
# df = pd.read_csv('https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz', sep='\t', error_bad_lines=False)

# Watches Category
df = pd.read_csv('https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz', sep='\t', error_bad_lines=False)
df

b'Skipping line 8704: expected 15 fields, saw 22\nSkipping line 16933: expected 15 fields, saw 22\nSkipping line 23726: expected 15 fields, saw 22\n'
b'Skipping line 85637: expected 15 fields, saw 22\n'
b'Skipping line 132136: expected 15 fields, saw 22\nSkipping line 158070: expected 15 fields, saw 22\nSkipping line 166007: expected 15 fields, saw 22\nSkipping line 171877: expected 15 fields, saw 22\nSkipping line 177756: expected 15 fields, saw 22\nSkipping line 181773: expected 15 fields, saw 22\nSkipping line 191085: expected 15 fields, saw 22\nSkipping line 196273: expected 15 fields, saw 22\nSkipping line 196331: expected 15 fields, saw 22\n'
b'Skipping line 197000: expected 15 fields, saw 22\nSkipping line 197011: expected 15 fields, saw 22\nSkipping line 197432: expected 15 fields, saw 22\nSkipping line 208016: expected 15 fields, saw 22\nSkipping line 214110: expected 15 fields, saw 22\nSkipping line 244328: expected 15 fields, saw 22\nSkipping line 248519: expected 15 fields,

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,3653882,R3O9SGZBVQBV76,B00FALQ1ZC,937001370,"Invicta Women's 15150 ""Angel"" 18k Yellow Gold ...",Watches,5,0,0,N,Y,Five Stars,Absolutely love this watch! Get compliments al...,2015-08-31
1,US,14661224,RKH8BNC3L5DLF,B00D3RGO20,484010722,Kenneth Cole New York Women's KC4944 Automatic...,Watches,5,0,0,N,Y,I love thiswatch it keeps time wonderfully,I love this watch it keeps time wonderfully.,2015-08-31
2,US,27324930,R2HLE8WKZSU3NL,B00DKYC7TK,361166390,Ritche 22mm Black Stainless Steel Bracelet Wat...,Watches,2,1,1,N,Y,Two Stars,Scratches,2015-08-31
3,US,7211452,R31U3UH5AZ42LL,B000EQS1JW,958035625,Citizen Men's BM8180-03E Eco-Drive Stainless S...,Watches,5,0,0,N,Y,Five Stars,"It works well on me. However, I found cheaper ...",2015-08-31
4,US,12733322,R2SV659OUJ945Y,B00A6GFD7S,765328221,Orient ER27009B Men's Symphony Automatic Stain...,Watches,4,0,0,N,Y,"Beautiful face, but cheap sounding links",Beautiful watch face. The band looks nice all...,2015-08-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960199,US,39508379,REH8UQZAXQS40,B00005OT9I,2153632,Xhilaration&#174; Silver Dial Bangle Watch,Watches,2,17,17,N,N,Great prize - but size??,It's a great watch - but you don't have a chan...,2001-12-13
960200,US,44704251,RJ14QPZEOI9P8,B00005QEM6,903687397,Armitron Men's Titanium 3 Eye Sport Analog Chr...,Watches,5,9,10,N,N,A very classy watch!,"Well worth the price. This is a light weight, ...",2001-11-08
960201,US,40571775,R2B1G5650WWFCE,B00005QEME,252821780,Men's Timex Internet Messenger Sport Watch,Watches,5,3,16,N,N,This is a great watch,"Dear Targeteers,<BR>This watch is exelent. it ...",2001-11-06
960202,US,44474855,R2MMGPUWXXOFI2,B00004YK0H,118389241,Energizer 393 Button Cell Battery,Watches,4,0,0,N,N,Now watt a minute here.,"In the old days, the common hearing battery in...",2001-04-05


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960204 entries, 0 to 960203
Data columns (total 15 columns):
marketplace          960204 non-null object
customer_id          960204 non-null int64
review_id            960204 non-null object
product_id           960204 non-null object
product_parent       960204 non-null int64
product_title        960202 non-null object
product_category     960204 non-null object
star_rating          960204 non-null int64
helpful_votes        960204 non-null int64
total_votes          960204 non-null int64
vine                 960204 non-null object
verified_purchase    960204 non-null object
review_headline      960197 non-null object
review_body          960056 non-null object
review_date          960200 non-null object
dtypes: int64(5), object(10)
memory usage: 109.9+ MB


In [12]:
analyzer = SentimentIntensityAnalyzer()

In [13]:
# Return dictionary of VADER sentiment scores for given sentence. Keys: 'neg', 'neu', 'pos', 'compound'
def sentiment_analyzer_scores(sentence, verbose=False):
    score = analyzer.polarity_scores(sentence)
    if verbose: print("{}".format(str(score)))
    
    return score
    
def sentiment_analyzer_scores_neg(sentence):
    return sentiment_analyzer_scores(sentence)['neg']

def sentiment_analyzer_scores_neu(sentence):
    return sentiment_analyzer_scores(sentence)['neu']

def sentiment_analyzer_scores_pos(sentence):
    return sentiment_analyzer_scores(sentence)['pos']

def sentiment_analyzer_scores_compound(sentence):
    return sentiment_analyzer_scores(sentence)['compound']

In [100]:
df_small = df.sample(n=20000)

# Deal with missing values for any of the columns we will be using
# df_small['star_rating'] = df_small['star_rating'].astype(int)
df_small['review_headline'] = df_small['review_headline'].fillna('.')
df_small['review_body'] = df_small['review_body'].fillna('.')
df_small['review_fulltext'] = df_small['review_headline'] + '. ' + df_small['review_body']

df_small['neg'] = df_small['review_fulltext'].apply(sentiment_analyzer_scores_neg)
df_small['neu'] = df_small['review_fulltext'].apply(sentiment_analyzer_scores_neu)
df_small['pos'] = df_small['review_fulltext'].apply(sentiment_analyzer_scores_pos)
df_small['compound'] = df_small['review_fulltext'].apply(sentiment_analyzer_scores_compound)

df_small.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 457332 to 572346
Data columns (total 20 columns):
marketplace          20000 non-null object
customer_id          20000 non-null int64
review_id            20000 non-null object
product_id           20000 non-null object
product_parent       20000 non-null int64
product_title        20000 non-null object
product_category     20000 non-null object
star_rating          20000 non-null int64
helpful_votes        20000 non-null int64
total_votes          20000 non-null int64
vine                 20000 non-null object
verified_purchase    20000 non-null object
review_headline      20000 non-null object
review_body          20000 non-null object
review_date          20000 non-null object
review_fulltext      20000 non-null object
neg                  20000 non-null float64
neu                  20000 non-null float64
pos                  20000 non-null float64
compound             20000 non-null float64
dtypes: float64(4), int64(

In [142]:
df_small['star_rating'].value_counts()

5    11871
4     3621
1     1743
3     1703
2     1062
Name: star_rating, dtype: int64

In [103]:
df_numer = df_small.select_dtypes(include='number')
df_numer.info()
df_numer

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 457332 to 572346
Data columns (total 9 columns):
customer_id       20000 non-null int64
product_parent    20000 non-null int64
star_rating       20000 non-null int64
helpful_votes     20000 non-null int64
total_votes       20000 non-null int64
neg               20000 non-null float64
neu               20000 non-null float64
pos               20000 non-null float64
compound          20000 non-null float64
dtypes: float64(4), int64(5)
memory usage: 1.5 MB


,customer_id,product_parent,star_rating,helpful_votes,total_votes,neg,neu,pos,compound
457332,49815900,481655606,5,0,0,0.000,0.542,0.458,0.9451
434668,37775518,688076877,5,0,0,0.000,1.000,0.000,0.0000
257077,17515479,118540561,5,0,0,0.000,0.589,0.411,0.6360
959686,40148190,655235200,5,0,0,0.000,0.667,0.333,0.9775
476647,15255826,49398577,5,0,0,0.000,0.744,0.256,0.9504
...,...,...,...,...,...,...,...,...,...
138878,53093486,911471808,5,0,0,0.121,0.720,0.159,0.3273
202392,10077685,221647894,1,0,1,0.289,0.410,0.301,0.0232
539090,36517701,819153640,5,2,2,0.014,0.712,0.273,0.9932
234147,9951879,345771086,4,0,0,0.055,0.770,0.175,0.9469


In [143]:
word_tokenizer = RegexpTokenizer(r'\w+')
df_small['review_wordlist'] = df_small['review_fulltext'].map(lambda x: word_tokenizer.tokenize(x.lower()))

In [144]:
df_small

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,...,verified_purchase,review_headline,review_body,review_date,review_fulltext,neg,neu,pos,compound,review_wordlist
457332,US,49815900,RVJTRSWKOWD56,B00590XSI2,481655606,DKNY Crystals Mother-of-pearl Dial Women's wat...,Watches,5,0,0,...,N,love it,I love it. I wear it every day. The silver i...,2014-06-26,love it. I love it. I wear it every day. The...,0.000,0.542,0.458,0.9451,"[love, it, i, love, it, i, wear, it, every, da..."
434668,US,37775518,RHXPB767LAIQ0,B000T9VK56,688076877,SE JT6216 21-Piece Watch Repair Kit,Watches,5,0,0,...,Y,I bought this so that I could replace my own ...,I bought this so that I could replace my own b...,2014-07-23,I bought this so that I could replace my own ....,0.000,1.000,0.000,0.0000,"[i, bought, this, so, that, i, could, replace,..."
257077,US,17515479,R35FUC607LKY25,B009459OYA,118540561,Camille Women's Chronograph Watch,Watches,5,0,0,...,Y,Five Stars,My son loved this watch!,2015-01-09,Five Stars. My son loved this watch!,0.000,0.589,0.411,0.6360,"[five, stars, my, son, loved, this, watch]"
959686,US,40148190,RPSUT1Z8NB9MX,B0002J26JW,655235200,WOMENS FOSSIL BIG TIC OCEAN WAVES WATCH FREE T...,Watches,5,0,0,...,N,This Watch Is So ME!!!,I just got this watch for Christmas and I LOVE...,2004-12-21,This Watch Is So ME!!!. I just got this watch ...,0.000,0.667,0.333,0.9775,"[this, watch, is, so, me, i, just, got, this, ..."
476647,US,15255826,R15CWWYSMQDXA9,B00AXUAXCM,49398577,"Hello Kitty Large Face Quartz Watch, Black Ban...",Watches,5,0,0,...,Y,Great deal!,This is the second time I have purchased a Hel...,2014-05-22,Great deal!. This is the second time I have pu...,0.000,0.744,0.256,0.9504,"[great, deal, this, is, the, second, time, i, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138878,US,53093486,R2AAK1ER06JNEE,B0080GQBTU,911471808,Panasonic CTL1616 Solar Rechargeable CTL 1616 ...,Watches,5,0,0,...,N,... came fully charged and ready for my Casio ...,My CTL1616 cell came fully charged and ready f...,2015-04-19,... came fully charged and ready for my Casio ...,0.121,0.720,0.159,0.3273,"[came, fully, charged, and, ready, for, my, ca..."
202392,US,10077685,R3A4RZKIZNIYFK,B001GUKPGC,221647894,Timex Expedition Shock Digital Compass Watch,Watches,1,0,1,...,Y,ok,didn't like the color,2015-02-21,ok. didn't like the color,0.289,0.410,0.301,0.0232,"[ok, didn, t, like, the, color]"
539090,US,36517701,R2U8GLTAV09SPO,B003SCNDQU,819153640,"Raymond Weil Women's 5799-STS-00995 ""Tango"" Di...",Watches,5,2,2,...,Y,Love my watch!,I was unfamiliar with the name Raymond Weil b...,2014-02-06,Love my watch!. I was unfamiliar with the name...,0.014,0.712,0.273,0.9932,"[love, my, watch, i, was, unfamiliar, with, th..."
234147,US,9951879,R3TLWAKM14LKB,B0058VJWTQ,345771086,Seiko Men's SNK795 Seiko 5 Automatic Stainless...,Watches,4,0,0,...,Y,Great watch for the price,"I have only had it for a week or so, but am ha...",2015-01-26,Great watch for the price. I have only had it ...,0.055,0.770,0.175,0.9469,"[great, watch, for, the, price, i, have, only,..."


In [145]:
df_small['review_wordlist'] = df_small['review_wordlist'].map(lambda x: ' '.join(x))

In [146]:
X = df_small.loc[:, ['helpful_votes', 'total_votes', 'neg', 'neu', 'pos', 'compound', 'review_wordlist']]
y = df_small.loc[:, 'star_rating']

In [147]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [148]:
# create our training data list - this is a list of strings, with each string being a review

clean_train_data = []

for train_data in X_train['review_wordlist']:
    clean_train_data.append(train_data)

In [149]:
# create test data list

clean_test_data = []

for test_data in X_test['review_wordlist']:
    clean_test_data.append(test_data)

In [150]:
# instantiate our CountVectorizer. This counts the number of appearances of all the words in our training data and
# eliminates common english stop words. 5000 max features works well for our purposes (tested various numbers). Our
# data is already preprocessed and tokenized manually earlier. ngram_range is 1,3, although all or nearly all our 
# features are single words

vectorizer = CountVectorizer(analyzer = "word",
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = 'english',
                             max_features = 5000,
                             ngram_range = (1, 3))

In [151]:
# fit our training data and test data lists to our count_vectorizer

train_data_features = vectorizer.fit_transform(clean_train_data)

test_data_features = vectorizer.transform(clean_test_data)

# convert to array

train_data_features = train_data_features.toarray()

In [152]:
# check shapes

train_data_features.shape, test_data_features.shape

((15000, 5000), (5000, 5000))

In [156]:
# I wanted check that the features corpus was as expected - removed print statement for readability

vocab = vectorizer.get_feature_names()
np.random.choice(vocab, 200)

array(['making', 'sized wrist', 'complications', 'really wanted',
       'watch white', 'operation', 'nice classy', 'looks works', '15 20',
       'penny', 'dial easy read', 'read', 'watch got', 'great features',
       'metallic', 'left', 'read watch', 'course', 'goes', 'original box',
       'minutes', 'arrive', 'watch set', 'keeps accurate time',
       'buy great', 'alert', 'watch need', 'switch', 'ocean',
       'unfortunately', 'buy new watch', 'watch lots', 'possible', '000',
       'uncomfortable', 'br br quality', 'automatics', 'watch nice',
       'ebay', 'como', 'exceptionally', 'stuck', 'watch problem',
       'casual', 'explained', 'hacking', 'nice looking', 'chance',
       'watch built', 'sealed', 'hole', 'contrast', 'sits', 'larger size',
       'watch design', 'reaction', 'fraction', 'continue', 'look really',
       'wearing watch', 'girlfriend loves', 'quality excellent',
       'nice touch', 'watch use', 'price little', 'nice br',
       'watch gorgeous', 'mother pe

In [157]:
y_train.value_counts()

5    8881
4    2729
1    1305
3    1300
2     785
Name: star_rating, dtype: int64

In [159]:
# Test out different classification algorithms
# Logistic Regression

logreg = LogisticRegression(fit_intercept=True, C=1e12, solver='liblinear', penalty='l2')
logreg.fit(X_train_resampled, y_train_resampled)

LogisticRegression(C=1000000000000.0, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [160]:
y_hat_test_lr = logreg.predict(X_test_numeric)
y_hat_train_lr = logreg.predict(X_train_resampled)

In [161]:
# We could subtract the two columns. If values or equal, difference will be zero. Then count number of zeros 
residuals = np.abs(y_train_resampled - y_hat_train_lr)
print(pd.Series(residuals).value_counts())
print(pd.Series(residuals).value_counts(normalize=True))

0    17799
1    17189
2     6772
3     2258
4      387
Name: star_rating, dtype: int64
0    0.400833
1    0.387096
2    0.152505
3    0.050850
4    0.008715
Name: star_rating, dtype: float64


In [162]:
residuals = np.abs(y_test - y_hat_test_lr)
print(pd.Series(residuals).value_counts())
print(pd.Series(residuals).value_counts(normalize=True))

0    2660
1    1510
2     479
3     278
4      73
Name: star_rating, dtype: int64
0    0.5320
1    0.3020
2    0.0958
3    0.0556
4    0.0146
Name: star_rating, dtype: float64


In [163]:
print(classification_report(y_train_resampled, y_hat_train_lr))

# Confusion Matrix for training set - Logistic Regression
cf_lr = confusion_matrix(y_train_resampled, y_hat_train_lr)
cf_lr

              precision    recall  f1-score   support

           1       0.45      0.60      0.51      8881
           2       0.32      0.22      0.26      8881
           3       0.33      0.19      0.24      8881
           4       0.33      0.36      0.34      8881
           5       0.49      0.64      0.56      8881

    accuracy                           0.40     44405
   macro avg       0.38      0.40      0.38     44405
weighted avg       0.38      0.40      0.38     44405



array([[5298, 1708,  873,  825,  177],
       [3824, 1977, 1329, 1312,  439],
       [1964, 1374, 1698, 2326, 1519],
       [ 478,  697,  896, 3180, 3630],
       [ 210,  516,  407, 2102, 5646]])

In [164]:
print(classification_report(y_test, y_hat_test_lr))

# Confusion Matrix for training set - Logistic Regression
cm_lr = confusion_matrix(y_test, y_hat_test_lr)
cm_lr

              precision    recall  f1-score   support

           1       0.47      0.63      0.54       438
           2       0.14      0.23      0.17       277
           3       0.16      0.17      0.17       403
           4       0.28      0.35      0.31       892
           5       0.81      0.65      0.72      2990

    accuracy                           0.53      5000
   macro avg       0.37      0.41      0.38      5000
weighted avg       0.60      0.53      0.56      5000



array([[ 274,   62,   55,   38,    9],
       [ 119,   65,   42,   40,   11],
       [  78,   81,   70,  101,   73],
       [  44,   77,  105,  315,  351],
       [  64,  185,  156,  649, 1936]])

In [ ]:
plot_confusion_matrix(logreg, X_test, y_test,
                                 display_labels=class_names,
                                 cmap=plt.cm.Blues,
                                 normalize=normalize)
    disp.ax_.set_title(title)

    print(title)
    print(disp.confusion_matrix)

In [165]:
# fit logistic regression model

lr = LogisticRegression(penalty = 'l2')
lr.fit(train_data_features, y_train)

/Users/megarock/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [166]:
lr.score(train_data_features, y_train)

0.8948

In [167]:
lr.score(test_data_features, y_test)

0.6556

In [168]:
# K Nearest Neighbors

# Instantiate KNeighborsClassifier
clf_knn = KNeighborsClassifier()

# Fit the classifier
clf_knn.fit(X_train_resampled, y_train_resampled)

# Predict on the training set
y_hat_knn_train = clf_knn.predict(X_train_resampled)

print(classification_report(y_train_resampled, y_hat_knn_train))

cm_knn_train = confusion_matrix(y_train_resampled, y_hat_knn_train)
cm_knn_train

              precision    recall  f1-score   support

           1       0.71      0.89      0.79      8881
           2       0.80      0.82      0.81      8881
           3       0.77      0.79      0.78      8881
           4       0.75      0.74      0.74      8881
           5       0.81      0.59      0.68      8881

    accuracy                           0.77     44405
   macro avg       0.77      0.77      0.76     44405
weighted avg       0.77      0.77      0.76     44405



array([[7934,  439,  295,  154,   59],
       [1143, 7251,  278,  133,   76],
       [ 853,  469, 7000,  318,  241],
       [ 487,  417,  591, 6574,  812],
       [ 691,  443,  928, 1599, 5220]])

In [169]:
y_hat_knn_test = clf_knn.predict(X_test_numeric)

print(classification_report(y_test, y_hat_knn_test))

cm_knn_test = confusion_matrix(y_test, y_hat_knn_test)
cm_knn_test

              precision    recall  f1-score   support

           1       0.26      0.46      0.33       438
           2       0.11      0.21      0.15       277
           3       0.11      0.21      0.14       403
           4       0.22      0.29      0.25       892
           5       0.81      0.49      0.61      2990

    accuracy                           0.41      5000
   macro avg       0.30      0.33      0.30      5000
weighted avg       0.56      0.41      0.46      5000



array([[ 200,   97,   81,   43,   17],
       [ 106,   58,   60,   40,   13],
       [ 100,   68,   84,   94,   57],
       [ 113,  102,  164,  257,  256],
       [ 247,  181,  387,  723, 1452]])

In [170]:
# Improve model performance by finding the best value for K

In [ ]:
clf_knn2 = KNeighborsClassifier()

# Fit the classifier
clf_knn2.fit(train_data_features, y_train)

# Predict on the training set
y_hat_knn2 = clf_knn2.predict(train_data_features)

print(classification_report(y_train, y_hat_knn2))

cm_knn2 = confusion_matrix(y_train, y_hat_knn2)
cm_knn2

In [121]:
# Decision Trees

clf_dt = DecisionTreeClassifier(criterion='entropy')

clf_dt.fit(X_train_resampled, y_train_resampled)

# Predict on the training set
y_hat_dt_train = clf_dt.predict(X_train_resampled)

cm_dt_train = confusion_matrix(y_train_resampled, y_hat_dt_train)
cm_dt_train

array([[8373,  506,    1,    0,    2],
       [  15, 8858,    1,    4,    4],
       [  25,  511, 8270,   19,   57],
       [  10,  259,   40, 8405,  168],
       [  24,  366,   49,  151, 8292]])

In [122]:
print(classification_report(y_train_resampled, y_hat_dt_train))

              precision    recall  f1-score   support

           1       0.99      0.94      0.97      8882
           2       0.84      1.00      0.91      8882
           3       0.99      0.93      0.96      8882
           4       0.98      0.95      0.96      8882
           5       0.97      0.93      0.95      8882

    accuracy                           0.95     44410
   macro avg       0.96      0.95      0.95     44410
weighted avg       0.96      0.95      0.95     44410



In [123]:
y_hat_dt_test = clf_dt.predict(X_test_numeric)

print(classification_report(y_test, y_hat_dt_test))

cm_dt_test = confusion_matrix(y_test, y_hat_dt_test)
cm_dt_test

              precision    recall  f1-score   support

           1       0.31      0.35      0.33       434
           2       0.12      0.28      0.17       261
           3       0.11      0.16      0.13       435
           4       0.21      0.25      0.23       881
           5       0.75      0.55      0.64      2989

    accuracy                           0.43      5000
   macro avg       0.30      0.32      0.30      5000
weighted avg       0.53      0.43      0.47      5000



array([[ 152,  125,   75,   43,   39],
       [  66,   74,   51,   30,   40],
       [  84,   80,   71,   98,  102],
       [  72,   94,  137,  221,  357],
       [ 112,  234,  317,  673, 1653]])

In [133]:
clf_dt2 = DecisionTreeClassifier(criterion='entropy')

# Fit the classifier
clf_dt2.fit(train_data_features, y_train)

# Predict on the training set
y_hat_dt2 = clf_dt2.predict(train_data_features)

print(classification_report(y_train, y_hat_dt2))

cm_dt2 = confusion_matrix(y_train, y_hat_dt2)
cm_dt2

              precision    recall  f1-score   support

           1       1.00      1.00      1.00      1412
           2       0.99      0.98      0.98       787
           3       0.99      0.94      0.97      1163
           4       0.98      0.89      0.93      2521
           5       0.96      1.00      0.98      9117

    accuracy                           0.97     15000
   macro avg       0.98      0.96      0.97     15000
weighted avg       0.97      0.97      0.97     15000



array([[1412,    0,    0,    0,    0],
       [   0,  773,    2,    1,   11],
       [   0,    8, 1095,    5,   55],
       [   0,    3,    3, 2234,  281],
       [   0,    0,    5,   39, 9073]])

In [124]:
# Bagged Trees

# Instantiate a BaggingClassifier
bagged_tree =  BaggingClassifier(DecisionTreeClassifier(criterion='gini', max_leaf_nodes=100), 
                                 n_estimators=20)

# Fit to the training data
bagged_tree.fit(X_train_resampled, y_train_resampled)

bagged_tree.score(X_train_resampled, y_train_resampled)

0.5003827966674173

In [125]:
bagged_tree.score(X_test_numeric, y_test)

0.4966

In [126]:
y_hat_bagged_train = bagged_tree.predict(X_train_resampled)

cm_bagged_train = confusion_matrix(y_train_resampled, y_hat_bagged_train)
cm_bagged_train

array([[5706, 1816,  810,  463,   87],
       [2096, 4922,  905,  728,  231],
       [1439, 1736, 3183, 1661,  863],
       [ 458, 1092, 1271, 2943, 3118],
       [ 273,  670,  762, 1709, 5468]])

In [127]:
print(classification_report(y_train_resampled, y_hat_bagged_train))

              precision    recall  f1-score   support

           1       0.57      0.64      0.61      8882
           2       0.48      0.55      0.51      8882
           3       0.46      0.36      0.40      8882
           4       0.39      0.33      0.36      8882
           5       0.56      0.62      0.59      8882

    accuracy                           0.50     44410
   macro avg       0.49      0.50      0.49     44410
weighted avg       0.49      0.50      0.49     44410



In [128]:
y_hat_bagged_test = bagged_tree.predict(X_test_numeric)

print(classification_report(y_test, y_hat_bagged_test))

cm_bagged_test = confusion_matrix(y_test, y_hat_bagged_test)
cm_bagged_test

              precision    recall  f1-score   support

           1       0.43      0.54      0.48       434
           2       0.14      0.34      0.20       261
           3       0.17      0.22      0.19       435
           4       0.24      0.27      0.25       881
           5       0.81      0.61      0.70      2989

    accuracy                           0.50      5000
   macro avg       0.36      0.40      0.36      5000
weighted avg       0.58      0.50      0.53      5000



array([[ 234,  119,   42,   29,   10],
       [  84,   90,   48,   31,    8],
       [  79,  104,   94,   83,   75],
       [  56,   99,  146,  238,  342],
       [  89,  214,  233,  626, 1827]])

In [129]:
# Random Forest

In [130]:
# Instantiate and fit a RandomForestClassifier
forest = RandomForestClassifier(n_estimators=100, max_depth = 10)
forest.fit(X_train_resampled, y_train_resampled)
forest.score(X_train_resampled, y_train_resampled)

0.6050439090294979

In [131]:
y_hat_forest = forest.predict(X_train_resampled)

cm_forest = confusion_matrix(y_train_resampled, y_hat_forest)
cm_forest

array([[6188, 1600,  620,  391,   83],
       [1372, 6058,  628,  626,  198],
       [1054, 1576, 4498, 1036,  718],
       [ 396,  971,  917, 4212, 2386],
       [ 257,  632,  681, 1398, 5914]])

In [132]:
print(classification_report(y_train_resampled, y_hat_forest))

              precision    recall  f1-score   support

           1       0.67      0.70      0.68      8882
           2       0.56      0.68      0.61      8882
           3       0.61      0.51      0.55      8882
           4       0.55      0.47      0.51      8882
           5       0.64      0.67      0.65      8882

    accuracy                           0.61     44410
   macro avg       0.60      0.61      0.60     44410
weighted avg       0.60      0.61      0.60     44410



In [133]:
y_hat_forest_test = forest.predict(X_test_numeric)

print(classification_report(y_test, y_hat_forest_test))

cm_forest_test = confusion_matrix(y_test, y_hat_forest_test)
cm_forest_test

              precision    recall  f1-score   support

           1       0.44      0.53      0.48       434
           2       0.16      0.37      0.22       261
           3       0.17      0.23      0.19       435
           4       0.25      0.28      0.26       881
           5       0.81      0.62      0.70      2989

    accuracy                           0.50      5000
   macro avg       0.36      0.40      0.37      5000
weighted avg       0.59      0.50      0.53      5000



array([[ 228,  122,   44,   29,   11],
       [  71,   97,   55,   28,   10],
       [  78,   99,  100,   82,   76],
       [  57,   94,  140,  243,  347],
       [  88,  198,  257,  595, 1851]])

In [117]:
# Grid Search / Random Search to find optimal parameters

In [118]:
# SVM / Linear SVC

In [96]:
# clf_svc = SVC(kernel='linear')
# clf_svc.fit(X_train_resampled, y_train_resampled)

# # Predict on the training set
# y_hat_svc = clf_svc.predict(X_train_resampled)

# print(classification_report(y_train_resampled, y_hat_svc))

# cm_svc = confusion_matrix(y_train_resampled, y_hat_svc)
# cm_svc


In [121]:
# Ensemble Classifiers

In [134]:
# Adaboost
# Instantiate an AdaBoostClassifier
clf_ab = AdaBoostClassifier()

clf_ab.fit(X_train_resampled, y_train_resampled)
clf_ab.score(X_train_resampled, y_train_resampled)

0.4098626435487503

In [135]:
y_hat_ab_train = clf_ab.predict(X_train_resampled)

cm_ab_train = confusion_matrix(y_train_resampled, y_hat_ab_train)
cm_ab_train

array([[5028, 2195, 1166,  433,   60],
       [3495, 2743, 1547,  877,  220],
       [1725, 1957, 2154, 1951, 1095],
       [ 541,  795, 1472, 2809, 3265],
       [ 239,  534,  738, 1903, 5468]])

In [136]:
print(classification_report(y_train_resampled, y_hat_ab_train))

              precision    recall  f1-score   support

           1       0.46      0.57      0.51      8882
           2       0.33      0.31      0.32      8882
           3       0.30      0.24      0.27      8882
           4       0.35      0.32      0.33      8882
           5       0.54      0.62      0.58      8882

    accuracy                           0.41     44410
   macro avg       0.40      0.41      0.40     44410
weighted avg       0.40      0.41      0.40     44410



In [137]:
y_hat_ab_test = clf_ab.predict(X_test_numeric)

print(classification_report(y_test, y_hat_ab_test))

cm_ab_test = confusion_matrix(y_test, y_hat_ab_test)
cm_ab_test

              precision    recall  f1-score   support

           1       0.43      0.52      0.47       434
           2       0.16      0.31      0.21       261
           3       0.16      0.23      0.19       435
           4       0.24      0.29      0.26       881
           5       0.81      0.62      0.70      2989

    accuracy                           0.50      5000
   macro avg       0.36      0.39      0.37      5000
weighted avg       0.58      0.50      0.53      5000



array([[ 227,  116,   51,   35,    5],
       [  84,   81,   59,   25,   12],
       [  82,   81,  101,   98,   73],
       [  54,   79,  146,  253,  349],
       [  83,  149,  258,  654, 1845]])

In [138]:
# Instantiate an GradientBoostingClassifier
clf_gb = GradientBoostingClassifier()

clf_gb.fit(X_train_resampled, y_train_resampled)
clf_gb.score(X_train_resampled, y_train_resampled)

0.4839450574195001

In [139]:
y_hat_gb_train = clf_gb.predict(X_train_resampled)

cm_gb_train = confusion_matrix(y_train_resampled, y_hat_gb_train)
cm_gb_train

array([[5145, 2113, 1120,  399,  105],
       [2073, 4454, 1361,  753,  241],
       [1237, 1983, 3049, 1713,  900],
       [ 410,  957, 1414, 3197, 2904],
       [ 228,  616,  761, 1630, 5647]])

In [140]:
print(classification_report(y_train_resampled, y_hat_gb_train))

              precision    recall  f1-score   support

           1       0.57      0.58      0.57      8882
           2       0.44      0.50      0.47      8882
           3       0.40      0.34      0.37      8882
           4       0.42      0.36      0.39      8882
           5       0.58      0.64      0.60      8882

    accuracy                           0.48     44410
   macro avg       0.48      0.48      0.48     44410
weighted avg       0.48      0.48      0.48     44410



In [141]:
y_hat_gb_test = clf_gb.predict(X_test_numeric)

print(classification_report(y_test, y_hat_gb_test))

cm_gb_test = confusion_matrix(y_test, y_hat_gb_test)
cm_gb_test

              precision    recall  f1-score   support

           1       0.45      0.51      0.48       434
           2       0.16      0.34      0.22       261
           3       0.17      0.25      0.20       435
           4       0.25      0.27      0.26       881
           5       0.80      0.62      0.70      2989

    accuracy                           0.50      5000
   macro avg       0.36      0.40      0.37      5000
weighted avg       0.58      0.50      0.54      5000



array([[ 223,  118,   54,   29,   10],
       [  76,   89,   62,   23,   11],
       [  74,   94,  108,   75,   84],
       [  41,   84,  166,  237,  353],
       [  84,  181,  257,  601, 1866]])

In [52]:
# Clustering

from sklearn.cluster import KMeans
from sklearn.metrics import calinski_harabasz_score

num_clusters = np.arange(2, 20)

# Set number of clusters at initialization time
for n in num_clusters:
    k_means = KMeans(n_clusters=n) 

    # Run the clustering algorithm
    k_means.fit(df_numer) 

    # Generate cluster index values for each row
    cluster_assignments = k_means.predict(df_numer)
    
    print(n, calinski_harabasz_score(df_numer, cluster_assignments))

2 59486.25585177998
3 77674.18325588538
4 96384.21852818847
5 114546.82979016086
6 127600.80444243301
7 141010.59498579273
8 150682.09351094894
9 161561.0248727341
10 166964.61084244298
11 172383.19385670617
12 179948.81991115178
13 182327.44955418463
14 185306.0671946048
15 187196.97618489747
16 187075.00888801456
17 186023.28848008186
18 185515.0192828231
19 184560.6592451715
20 182653.62410723022
21 177354.84392849842
22 175636.95905210843
23 172503.3659090606
24 170951.62031425143
25 169277.9078372501
26 167893.9965466513
27 166426.02562539277
28 163768.2050005787
29 164446.90279474566
30 163345.56115803414
31 163941.37842270927
32 161616.26932644393
33 162014.7044419554
34 162781.62265305087
35 163483.25020053884
36 164121.61442848394
37 165910.86684445763
38 166263.34904917312
39 166967.81410185085
40 169879.35195707544
41 170036.8446218407
42 172317.16871043303
43 173225.85042507053
44 176405.45462941405
45 175951.96694708266
46 179420.62087730106
47 181230.39342520965
48 181994

In [53]:
cluster_assignments

array([20, 48, 43, ..., 40, 34, 23], dtype=int32)

In [54]:
k_means.labels_

array([20, 48, 43, ..., 40, 34, 23], dtype=int32)

In [55]:
# This code builds on the previous example

# Note that we could also pass in k_means.labels_ instead of cluster_assignments
print(calinski_harabasz_score(df_numer, cluster_assignments))

180807.4371004135
